# Step 1 - Import library, upload data, data exploration
### 1.1. Import library
- Import những thư viện tự viết là chính
- Thêm 1 số thư viện cần thì nên làm

In [1]:
# 1. Import library
import utilis.step1ml_utilis_summaryPre_processing as sum
import utilis.step2ml_utils_feature_selection as feature
import utilis.step3ml_utils_model_design_testing_classification as classi
import utilis.step4ml_utils_model_design_testing_explainability as tests
import utilis.step5ml_utils_model_design_testing_visualize_models as visu
import utilis.step6ml_utils_model_design_testing_geospatial_analysis as ana
import utilis.step7ml_utilis_NLP_Vietnamese as nlp
import utilis.step8ml_utilis_NLP as nlp_model
import utilis.step9ml_Text_Kmeans_Clustering as pys
import warnings
warnings.filterwarnings("ignore")


### 1.2. Upload Data
- Có hàm viết về upload file => hạn chế dùng trực tiếp hàm môi trường jupyter notebook

In [2]:
# 2. Upload data => name file: PNJ_Khao_sat.xlsx
import  pandas as pd
dir_file = "data/PNJ_Khao_sat.xlsx"
names = ['ID','Start_time','Completion_time','Email','Posision','Area','Name','Question','Reply','Group_relply']
df = sum.loadData(file_dir=dir_file, names=names)
#df = pd.read_excel(dir_file)
df.head()

,ID,Start_time,Completion_time,Email,Posision,Area,Name,Question,Reply,Group_relply
0,1,2021-11-04 14:49:35,2021-11-04 14:55:46,ai.dtn@pnj.com.vn,CHT,MTR,ĐỖ THỊ NHƯ ÁI,1,TVV Không nắm được nội dung quy định này và kh...,NaN
1,2,2021-11-04 14:51:46,2021-11-04 14:58:00,thanh.tt02@pnj.com.vn,CHT,DNB,Trần Thiện Thanh,1,TVV Không nắm được nội dung quy định này và kh...,NaN
2,3,2021-11-04 14:54:50,2021-11-04 15:01:28,tran.lhb@pnj.com.vn,CHT,TN,Lữ Hoàng Bảo Trân,1,Scheme dài và nhiều nội dung quá nên không nhớ...,NaN
3,4,2021-11-04 14:51:28,2021-11-04 15:02:52,quang.dd@pnj.com.vn,CHT,MTR,Dương Đệ Quảng,1,Scheme dài và nhiều nội dung quá nên không nhớ...,NaN
4,5,2021-11-04 14:54:55,2021-11-04 15:11:01,canh.nd@pnj.com.vn,CHT,MBC,Nguyễn Đức Cảnh,1,Scheme dài và nhiều nội dung quá nên không nhớ...,NaN


In [3]:
# Nhìn biểu đồ xem dự đoán xem dữ liệu continious hay category
sum.df_overview(df=df, choose="visualization")

 Categerocial   Numerical   NaN 


(Text(0.5, 1.0, 'Dataset Overview'), None)

### Reply
- Cần khai thác thông tin cần thiệt đối với bài toán NLP
- Xây dựng các predict để chia xem có ý nghĩa gì với model không ?
- Áp dụng thuật toán Clustering
- Đối với thuộc tính nội dung:
    + Thêm thông tin bổ sung cho bài toán
    + Cần phân tích thêm xem nội dung này có chia theo nhóm và thực hiện được dự đoán nhóm theo category thêm cho model và có ý nghĩa không ?
- Các bước thực hiện
    + Cần xóa những thông tin không cần thiết của bài toán như số, những ký tự đặng biệt thông qua hàm process_nlp_vietnamese
    + Cần làm 1 file liên quan đến stop word để thực hiện lấy những từ có giá trị trong predict nhà
    + Xây dựng hàm thực hiện lọc từ có nghĩa của Vietnamese để thực hiện NLP
    + Dùng pyspark để phân cụm bài toán NLP

In [4]:
# Lọc dữ liệu và loại bỏ những dữ liệu không cần thiết ra khỏi NLP
df_Reply = nlp.process_nlp_vietnamese(df=df, lst_text='Reply',columns_='Reply')
df_Reply

,Reply
0,tvv không nắm được nội_dung quy_định này và kh...
1,tvv không nắm được nội_dung quy_định này và kh...
2,scheme dài và nhiều nội_dung quá nên không nhớ...
3,scheme dài và nhiều nội_dung quá nên không nhớ...
4,scheme dài và nhiều nội_dung quá nên không nhớ...
...,...
2185,bên đối_tác qua lấy gọi qua check tại ch để ti...
2186,cửa_hàng trong tttm lớn khi bên đtvc đến hay y...
2187,không
2188,đối_với th khách_hàng ko lấy hàng ch check trự...


In [5]:
# Lấy dữ liệu theo số lần lặp để thực hiện lấy những cụm từ có ý nghĩa trong bài toán của chúng ta
word_again = nlp.groupby_word_txt(df=df, lst_text='Reply', columns_='Reply')
word_again

,Num
Reply,
không,1392
có,1085
và,960
chương_trình,912
nên,867
...,...
quảng_cáo,1
quẹt,1
qđ,1


In [6]:
# word_again.to_excel('stop_word.xlsx')

In [7]:
# Chuyển đổi dữ liệu từ dataframe sang pyspark
pys_df =  pys.upload_dataframe(df=df_Reply)
pys_df.show(5, truncate=True)

+--------------------+
|               Reply|
+--------------------+
|tvv không nắm đượ...|
|tvv không nắm đượ...|
|scheme dài và nhi...|
|scheme dài và nhi...|
|scheme dài và nhi...|
+--------------------+
only showing top 5 rows



In [8]:
# Thực hiện pipeline cho bài toán
pre_data = pys.pipeline_psy(df_pys=pys_df, lst_inputCol='Reply')

In [9]:
# Chọn k tốt nhất cho dữ liệu => Thực hiện bài toán
k_list, wsse_list, sihoutte_list = pys.choose_k_in_kmeans(pre_data=pre_data,k_from=2,k_to=30)

wsse_str:   with k =2- Set sum of Squared Errors284222.61793610774
 sil_str:  With k=2- Sihoutte = 0.6783982307326879

wsse_str:   with k =2- Set sum of Squared Errors284222.61793610774
with k =3- Set sum of Squared Errors277184.26253992587
 sil_str:  With k=2- Sihoutte = 0.6783982307326879
With k=3- Sihoutte = 0.5349661614040387

wsse_str:   with k =2- Set sum of Squared Errors284222.61793610774
with k =3- Set sum of Squared Errors277184.26253992587
with k =4- Set sum of Squared Errors274234.5682543672
 sil_str:  With k=2- Sihoutte = 0.6783982307326879
With k=3- Sihoutte = 0.5349661614040387
With k=4- Sihoutte = 0.5202736216859524

wsse_str:   with k =2- Set sum of Squared Errors284222.61793610774
with k =3- Set sum of Squared Errors277184.26253992587
with k =4- Set sum of Squared Errors274234.5682543672
with k =5- Set sum of Squared Errors259785.65347780657
 sil_str:  With k=2- Sihoutte = 0.6783982307326879
With k=3- Sihoutte = 0.5349661614040387
With k=4- Sihoutte = 0.52027362168595

In [10]:
# Trực quan hóa dữ liệu
pys.visualization_k(k_list=k_list, sihoutte_list=sihoutte_list)

([<matplotlib.lines.Line2D at 0x1bd27e346c8>], None)

In [11]:
# Xem những chỉ số quan trọng đến bài toán clustering => Chọn k = 3 cụm
wsse, sihouette, lst,pre, df_psy = pys.model_kmeans(pre_data=pre_data, k=25)

Within Set Sum of Squared Errors = 220226.9921315328
Sihoutte = 0.1349747816763849
Cluster Centers: 
+----------+-----+
|prediction|count|
+----------+-----+
|        12|   66|
|        22|   38|
|         1| 1270|
|        13|  167|
|        16|    4|
|         6|   39|
|         3|    5|
|        20|    1|
|         5|   13|
|        19|   47|
|        15|  210|
|        17|   66|
|         9|    3|
|         4|    1|
|         8|   22|
|        23|    1|
|         7|    1|
|        10|   96|
|        24|   10|
|        21|    1|
+----------+-----+
only showing top 20 rows



In [12]:
# Giá trị dự đoán của cụm
df_content = pys.choose_select_wordText(pre=pre, text='Reply')
df_content
df_content = df_content.rename(columns={"Reply": "Reply", "prediction": "prediction_Reply"})
df_content

,Reply,prediction_Reply
0,tvv không nắm được nội_dung quy_định này và kh...,2
1,tvv không nắm được nội_dung quy_định này và kh...,2
2,scheme dài và nhiều nội_dung quá nên không nhớ...,2
3,scheme dài và nhiều nội_dung quá nên không nhớ...,1
4,scheme dài và nhiều nội_dung quá nên không nhớ...,1
...,...,...
2185,bên đối_tác qua lấy gọi qua check tại ch để ti...,1
2186,cửa_hàng trong tttm lớn khi bên đtvc đến hay y...,15
2187,không,1
2188,đối_với th khách_hàng ko lấy hàng ch check trự...,15


In [13]:
df_content.isna().sum()

Reply               0
prediction_Reply    0
dtype: int64

In [14]:
# Trực quan hóa dữ liệu và xem cụm tư phân loại
pys.visualization_wordcloud(df_content, text_='Reply',prediction='prediction_Reply',k_from=0,k_to=24)

Keyword Kmean Class: 0
Keyword Kmean Class: 1
Keyword Kmean Class: 2
Keyword Kmean Class: 3
Keyword Kmean Class: 4
Keyword Kmean Class: 5
Keyword Kmean Class: 6
Keyword Kmean Class: 7
Keyword Kmean Class: 8
Keyword Kmean Class: 9
Keyword Kmean Class: 10
Keyword Kmean Class: 11
Keyword Kmean Class: 12
Keyword Kmean Class: 13
Keyword Kmean Class: 14
Keyword Kmean Class: 15
Keyword Kmean Class: 16
Keyword Kmean Class: 17
Keyword Kmean Class: 18
Keyword Kmean Class: 19
Keyword Kmean Class: 20
Keyword Kmean Class: 21
Keyword Kmean Class: 22
Keyword Kmean Class: 23


(<Figure size 1500x500 with 1 Axes>,
 (-0.5, 399.5, 199.5, -0.5),
 None)

In [15]:
import pandas as pd
lst_concat = [df, df_content]
df_new = pd.concat(lst_concat, axis=1)
df_new.head()

,ID,Start_time,Completion_time,Email,Posision,Area,Name,Question,Reply,Group_relply,Reply,prediction_Reply
0,1,2021-11-04 14:49:35,2021-11-04 14:55:46,ai.dtn@pnj.com.vn,CHT,MTR,ĐỖ THỊ NHƯ ÁI,1,TVV Không nắm được nội dung quy định này và kh...,NaN,tvv không nắm được nội_dung quy_định này và kh...,2
1,2,2021-11-04 14:51:46,2021-11-04 14:58:00,thanh.tt02@pnj.com.vn,CHT,DNB,Trần Thiện Thanh,1,TVV Không nắm được nội dung quy định này và kh...,NaN,tvv không nắm được nội_dung quy_định này và kh...,2
2,3,2021-11-04 14:54:50,2021-11-04 15:01:28,tran.lhb@pnj.com.vn,CHT,TN,Lữ Hoàng Bảo Trân,1,Scheme dài và nhiều nội dung quá nên không nhớ...,NaN,scheme dài và nhiều nội_dung quá nên không nhớ...,2
3,4,2021-11-04 14:51:28,2021-11-04 15:02:52,quang.dd@pnj.com.vn,CHT,MTR,Dương Đệ Quảng,1,Scheme dài và nhiều nội dung quá nên không nhớ...,NaN,scheme dài và nhiều nội_dung quá nên không nhớ...,1
4,5,2021-11-04 14:54:55,2021-11-04 15:11:01,canh.nd@pnj.com.vn,CHT,MBC,Nguyễn Đức Cảnh,1,Scheme dài và nhiều nội dung quá nên không nhớ...,NaN,scheme dài và nhiều nội_dung quá nên không nhớ...,1


In [16]:
df_new.to_excel('data/pnj_khao_sat_final.xlsx')